<img style="float: center;" src="images/CI_horizontal.png" width="1000">
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

Rayid Ghani, Frauke Kreuter, Julia Lane, Brian Kim, Adrianne Bradford, Alex Engler, Nicolas Guetta Jeanrenaud, Graham Henke, Daniela Hochfellner, Clayton Hunter, Avishek Kumar, Jonathan Morgan, Ursula Kaczmarek, Benjamin Feder, Ekaterina Levitskaya, Lina Osorio-Copete, Tian Lou.

# Data Preparation for Clustering
------

## Introduction

In this notebook, we will use Ohio Unemployment Insurance (UI) wage records to create an employer file, `employer_all_new.csv` and use it to run a clustering model in [Unsupervised Machine Learning](Unsupervised_ML.ipynb) notebook. We will focus on all the employers present in Ohio UI data during the third quarter of 2013 and create the following variables:

- **Total employment**: number of paid employees each employer has during 2013Q3
- **Total payroll**: the sum of wages an employer paid to all of its employees during 2013Q3
- **Average payroll**: total payroll divided by total employment
- **Earnings at the bottom 25th percentile**: the bottom 25% earnings within each employer during 2013Q3
- **Earnings at the top  25th percentile**: the top 25% earnings within each employer during 2013Q3
- **Total full quarter employees**: number of employees worked during 2013Q2, 2013Q3, and 2013Q4 within each employer
- **Total payroll for full quarter employees**: the sum of wages an employer paid to all of its full quarter employees during 2013Q3
- **Average payroll per full quarter employee**: total payroll for full quarter employees divided by total full quarter employees
- **Separation growth rate**: percentage change in job separations within an employer from 2013Q2 to 2013Q3
- **Hiring growth rate**: percentage change in new hires within an employer from 2013Q2 to 2013Q3
- **Employment growth rate**: percentage change in an employer's size from 2013Q2 to 2013Q3
    

## Python Setup
First, let us run the cell below to import relevant libraries and establish our connection to the database.

In [ ]:
# pandas-related imports
import pandas as pd

# Numpy
import numpy as np

# database interaction imports
import sqlalchemy

# import viz 
import matplotlib.pyplot as plt

#import clustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
import scipy.cluster.hierarchy as sch

In [ ]:
# to create a connection to the database, 
# we need to pass the name of the database and host of the database

host = 'stuffed.adrf.info'
DB = 'appliedda'

connection_string = "postgresql://{}/{}".format(host, DB)
conn = sqlalchemy.create_engine(connection_string)

It will take us a long time to run the code if we pull all the Ohio UI data from `data_ohio_olda_2018`. Therefore, we have created a subset of the `oh_ui_wage_by_employer` table, which only contains the data we need for this analysis (2013Q2-2013Q4 UI data) and for checkpoint (2013Q1). We call this table `oh_2013q3` and have saved it in schema `ada_20_osu`.

The code used to generate the `oh_2013q3` table is available below.

    create table ada_20_osu.oh_2013q3 as 
    select *, format('%%s-%%s-1', year, quarter*3-2)::date as job_yr_q
    from data_ohio_olda_2018.oh_ui_wage_by_employer 
    where year = '2013';

> We need information on employers from 2013Q2 and 2013Q4 to calculate full-quarter employment statistics, as well as hiring, employment, and separation rates.
> In the checkpoints, you will be using 2013Q1-2013Q3 data to calculate the same statistics.


## 1. Explore the Data

Before we generate employer-level statistics, let's explore our table a little bit. This will give us some context about both the employers and the employees in the `oh_2013q3` table, as well as the overall labor market in Ohio.
> Checking basic counts can also serve as a sanity check to make sure you properly subsetted your table.

In [ ]:
#see table first
qry = '''
select * 
from ada_20_osu.oh_2013q3
limit 5
'''
pd.read_sql(qry, conn)

In [ ]:
#number of distinct employers in table
qry = '''
select count(distinct(employer)) 
from ada_20_osu.oh_2013q3
'''
pd.read_sql(qry, conn)

In [ ]:
# number of distinct employers and employees with entries in 2013Q3
qry = '''
select count(distinct(employer)) as employer_num, count(distinct(ssn_hash)) as employee_num 
from ada_20_osu.oh_2013q3
where quarter = 3 and year = '2013'
'''
pd.read_sql(qry, conn)

In [ ]:
# number of employers by quarter
qry = '''
select quarter, count(distinct(employer)) as employer_num
from ada_20_osu.oh_2013q3
group by quarter
order by quarter
'''
pd.read_sql(qry, conn)

<font color=red><h3> Checkpoint 1: Sanity Check </h3></font> 

How many employees have earnings entries in 2013Q2? Does this number differ significantly from the amount of employees in 2013Q3?

## 2. Calculate Employment and Earnings Statistics 

### Employment and Earnings of All Paid Employees within Each Employer
Now that we have a better sense of the amount of employers and employees in 2013Q3, we can start to aggregate earnings and the number of employees by employer to generate our desired employer-level statistics. In this section, we will calculate: 

<il>
<li>a. <b>Total employment</b>: <code>num_employed</code></li> 
<li>b. <b>Total payroll</b>: <code>total_earnings</code> </li>
<li>c. <b>Average payroll</b>: <code>avg_earnings</code> </li>
<li>d. <b>Earnings at the bottom 25 percentile</b>: <code>bottom_25_pctile</code> </li>
<li>e. <b>Earnings at the top 25 percentile</b>: <code>top_25_pctile</code> </li>
</il>  
<p>
     
We can find this set of measures by only using 2013Q3 data and some simple manipulations. To find earnings percentiles, we just need to use `PERCENTILE_DISC() WITHIN GROUP (ORDER BY WAGES) AS`.

In [ ]:
#total earnings, number of employees, average earnigns, top 25% earnings, bottom 25% earnings
#by employer in 2013Q3
qry = '''
select employer, naics_3_digit, count(ssn_hash) as num_employed, sum(wages) as total_earnings, 
    sum(wages)/count(ssn_hash) as avg_earnings, year, quarter,
    percentile_disc(0.25) within group (order by wages) as bottom_25_pctile,
    percentile_disc(0.75) within group (order by wages) as top_25_pctile
from ada_20_osu.oh_2013q3
where quarter = 3 and year = '2013'
group by employer, naics_3_digit, year, quarter;
'''
employer_df = pd.read_sql(qry, conn)

In [ ]:
# see employer_df
employer_df.head()

### Employment and Earnings of Full-Quarter Employees within Each Employer

We define **full-quarter employees** as workers who have worked for the same employers during time *t-1*, *t*, and *t+1*. To find these workers, we will join three copies of the same `oh_2013q3` table with a SQL `WHERE` clause to confirm the person was employed in three consecutive quarters and worked for the same employer. In this case, we are looking at 2013Q2, 2013Q3, and 2013Q4 to determine if someone experienced full-quarter employment in 2013Q3. By aggregating on the employer level, we will find these measures: a. **number of full quarter employees**: `full_quarter_num`; b. **total payroll for full quarter employees**: `full_quarter_earnings`; c. **average earnings per full quarter employees**: `full_quarter_avg_earnings`.
 

In [ ]:
#Count the number of people who have worked for the same employer during 2013Q2, 2013Q3, and 2013Q4
#The total wages each employer paid for full quarter employees
#And the average earnings each employer paid for full quarter employeres
qry = '''
select a.employer, count(distinct(a.ssn_hash)) as full_quarter_num, sum(a.wages) as full_quarter_earnings, 
    sum(a.wages)/count(distinct(a.ssn_hash)) as full_quarter_avg_earnings
from ada_20_osu.oh_2013q3 a, ada_20_osu.oh_2013q3 b, ada_20_osu.oh_2013q3 c
where a.ssn_hash = b.ssn_hash and a.employer=b.employer and
a.ssn_hash = c.ssn_hash and a.employer = c.employer and a.job_yr_q = (b.job_yr_q - '3 month'::interval)::date and 
a.job_yr_q = (c.job_yr_q + '3 month'::interval)::date 
group by a.employer
'''

full_quarter_df = pd.read_sql(qry, conn)

In [ ]:
# see full_quarter_df
full_quarter_df.head()

<font color=red><h3> Checkpoint 2: Generate Employment and Earnings Statistics for 2013Q2 Employers  </h3></font> 

Try to generate some employer-level characteristics for 2013Q2 employers:

1. Calculate total employment, total payroll, average payroll, bottom 25% earnings, top 25% earnings for 2013Q2 employers. 
2. Calculate the number of full-quarter employees, total payroll for full-quarter employees, average earnings per full-quarter employee for 2013Q2 employers.

## 3. Employment, Separation, and Hiring Growth Rates

By looking at by employer UI data across quarters, we can observe the change in employment for each employer. In addition, since we know which person worked for which employer at what time, we can calculate how many workers left an employer during a quarter and how many new workers an employer hired during a quarter. 

However, should we directly use these numbers in our model? Probably not. The magnitude of job destructions and job creations largely depend on the size of a firm. Larger firms usually have larger numbers of job separations and new hires during each quarter than smaller firms, but it doesn't imply employment in larger firms are less stable. 

Therefore, we need to normalize the number of employment, the number of job separations, and the number of new hires so that employers of different sizes become more comparable with each other. We use the following function from <a href='https://academic.oup.com/qje/article-abstract/107/3/819/1873525'>Davis and Haltiwanger (1992)</a> to calculate 1) employment growth rate: `emp_rate`; 2) separation growth rate: `sep_rate`; 3) hire growth rate: `hire_rate`.

$$ g_{et}=\frac{2(x_{et} - x_{e,t-1})}{(x_{et} + x_{e,t-1})} $$

In this function, $g_{et}$ represents employment/separation/hire growth rate of employer $e$ at time $t$. $x_{et}$ and $x_{e,t-1}$ are employer $e$'s employment/separation/hire at time $t$ and $t-1$, respectively. According to Davis and Haltiwanger (1992):

"*This growth rate measure is symmetric about zero, and it lies in the closed interval [-2,2] with deaths (births) corresponding to the left (right) endpoint. A virtue of this measure is that it facilitates an integrated treatment of births, deaths, and continuing establishments in the empirical analysis.*"

In other words, a firm with a $ g_{et} = 2$ is a new firm, while a firm with a $ g_{et} = -2$ is a a firm that exited the economy.
    
> Why do the two endpoints represent firms' deaths and births? Calculate the value of $g_{et}$ when $x_{et}=0$ and when $x_{e,t-1}=0$ and see what you get.

### Employment Growth Rate

Let's calculate the employment growth rate first. In the previous step, we have already calculated total number of employment within each employer in 2013Q3, `num_employed`. Here, we need to get the total number of employment within each employer in 2013Q2, `num_employed_q2`.

In [ ]:
#number of employees during 2013Q2
qry = '''
select employer, count(ssn_hash) as num_employed_q2
from ada_20_osu.oh_2013q3
where quarter = 2 and year = '2013'
group by employer
'''
emp_q2_df = pd.read_sql(qry, conn)

In [ ]:
# Take a look at the table
emp_q2_df.head()

In [ ]:
# Let's merge it with employer_df
employer_df = employer_df.merge(emp_q2_df, on = 'employer', how = 'left')

In [ ]:
#check the new variable
employer_df[['num_employed','num_employed_q2']].describe()

We can see that `num_employed_q2` has some missing values. This is probably because some employers are new in 2013Q3. Let's fill in these missing values with zero.

In [ ]:
employer_df['num_employed_q2'] = employer_df['num_employed_q2'].fillna(0)

employer_df['num_employed_q2'].describe()

Now we have both `num_employed` and `num_employed_q2` in our DataFrame. Let's calculate employment growth rate based on the normalization function.

In [ ]:
#growth rates of employment
employer_df['emp_rate'] = 2 * (employer_df['num_employed'] - 
                               employer_df['num_employed_q2']) / (employer_df['num_employed'] +
                                                                  employer_df['num_employed_q2'])

In [ ]:
employer_df['emp_rate'].describe()

### Separation Growth Rate

Next, let's calculate separataion growth rate. Similar to the employment growth rate, we will need the number of job separations for each employer in both 2013Q2 and 2013Q3. To find the number of employees that separated from their employer in 2013Q2(Q3), in practice, we take `employer` and `ssn_hash` entries from consecutive quarters and if a person is in the first but is not in the next quarter, we assume this individual was separated from their job. The process followed is this:
- Find the `ssn_hash` and `employer` for every individual that had earnings in 2013Q2(Q3)
- Find the `ssn_hash` and `employer` for every individual that had earnings in 2013Q3(Q4)
- Left join 2013Q3 to 2013Q2 (2013Q3 to 2013Q4) to include an indicator (`q3` or `q4`) of whether they were employed in 2013Q3(Q4) (NULL if not). If the person showed in 2013Q2 but not 2013Q3, it implies this person left their employer during 2013Q2. 
- Count the amount of `ssn_hash` values per `employer` where the indicator (`q3` or `q4`) is NULL

In [ ]:
# Number of people left their jobs in 2013Q2 by employer
qry = '''
select employer,count(distinct(ssn_hash)) as nsep_q2 from
	(select a.employer,a.ssn_hash,a.quarter as q2,b.quarter as q3 from
    	(select employer, ssn_hash, quarter from ada_20_osu.oh_2013q3
     	where quarter = 2 and year = '2013') a
	left join 
    	(select employer, ssn_hash, quarter from ada_20_osu.oh_2013q3
     	where quarter = 3 and year = '2013') b
	on a.employer=b.employer and a.ssn_hash=b.ssn_hash) c
where c.q3 is null
group by employer;
'''
nsep_q2_df = pd.read_sql(qry, conn)

In [ ]:
nsep_q2_df.head()

In [ ]:
# Number of people left their jobs in 2013Q3 by employer
qry = '''
select employer,count(distinct(ssn_hash)) as nsep_q3 from
	(select a.employer,a.ssn_hash,a.quarter,b.quarter as q4 from
    	(select employer, ssn_hash, quarter from ada_20_osu.oh_2013q3
     	where quarter = 3 and year = '2013') a
	left join 
    	(select employer, ssn_hash, quarter from ada_20_osu.oh_2013q3
     	where quarter = 4 and year = '2013') b
	on a.employer=b.employer and a.ssn_hash=b.ssn_hash) c
where c.q4 is null
group by employer;
'''
nsep_q3_df = pd.read_sql(qry, conn)

In [ ]:
nsep_q3_df.head()

Similarly, let's merge these two DataFrame with `employer_df` and fill in missing values with zero. Then we can use the normalization function to calculate separation growth rate.

In [ ]:
# Merge DataFrames
employer_df = employer_df.merge(nsep_q2_df, on = 'employer', how = 'left')

employer_df = employer_df.merge(nsep_q3_df, on = 'employer', how = 'left')

In [ ]:
# Fill missing values with zero
employer_df[['nsep_q2', 'nsep_q3']] = employer_df[['nsep_q2', 'nsep_q3']].fillna(0)

In [ ]:
# Checking nulls
employer_df.isna().sum()

In [ ]:
#growth rates of job separation
employer_df['sep_rate'] = 2 * (employer_df['nsep_q3'] - 
                               employer_df['nsep_q2']) / (employer_df['nsep_q2'] +
                                                          employer_df['nsep_q3'])

In [ ]:
# see separation growth rate distribution
employer_df['sep_rate'].describe()

### Hire Growth Rate

Finally, to find the number of employees hired by each employer during 2013Q3, we follow a similar protocol as above. However, instead of counting those in the previous quarter who were not in the current quarter, we will count all those in the current quarter who did not have earnings for the same employer in the previous quarter. In other words, instead of using `LEFT JOIN`, we will be using `RIGHT JOIN` between `ssn_hash` by `employer`.

In [ ]:
# Number of people got their jobs in 2013Q2, by employer
qry = '''
select employer,count(distinct(ssn_hash)) as nhire_q2 from
	(select b.employer,b.ssn_hash,a.quarter as q1,b.quarter as q2 from
    	(select employer, ssn_hash, quarter from ada_20_osu.oh_2013q3
     	where quarter = 1 and year = '2013') a
	right join 
    	(select employer, ssn_hash, quarter from ada_20_osu.oh_2013q3
     	where quarter = 2 and year = '2013') b
	on a.employer=b.employer and a.ssn_hash=b.ssn_hash) c
where c.q1 is null
group by employer;
'''
nhire_q2_df = pd.read_sql(qry, conn)

In [ ]:
nhire_q2_df.head()

In [ ]:
# Number of people got their jobs in 2013Q3, by employer
qry = '''
select employer,count(distinct(ssn_hash)) as nhire_q3 from
	(select b.employer,b.ssn_hash,a.quarter as q2,b.quarter as q3 from
    	(select employer, ssn_hash, quarter from ada_20_osu.oh_2013q3
     	where quarter = 2 and year = '2013') a
	right join 
    	(select employer, ssn_hash, quarter from ada_20_osu.oh_2013q3
     	where quarter = 3 and year = '2013') b
	on a.employer=b.employer and a.ssn_hash=b.ssn_hash) c
where c.q2 is null
group by employer;
'''
nhire_q3_df = pd.read_sql(qry, conn)

In [ ]:
nhire_q3_df.head()

In [ ]:
# Merge DataFrames
employer_df = employer_df.merge(nhire_q2_df, on = 'employer', how = 'left')

employer_df = employer_df.merge(nhire_q3_df, on = 'employer', how = 'left')

In [ ]:
# Fill missing values with zero
employer_df[['nhire_q2', 'nhire_q3']] = employer_df[['nhire_q2', 'nhire_q3']].fillna(0)

In [ ]:
# Checking nulls
employer_df.isna().sum()

In [ ]:
#growth rates of new hires
employer_df['hire_rate'] = 2 * (employer_df['nhire_q3'] - 
                                employer_df['nhire_q2']) / (employer_df['nhire_q2'] +
                                                            employer_df['nhire_q3'])

In [ ]:
# see hiring growth rate distribution
employer_df['hire_rate'].describe()

In [ ]:
employer_df.info()

<font color=red><h3> Checkpoint 3: Find Growth Rates  </h3></font> 

Find hiring, separation, and employment growth rates for all employers in 2013Q2.

## 4. Final Data Manipulation

At this point, we've created all of our desired measures. To get the dataset ready for clustering, we have to complete a few small tasks first, such as dropping unnecessary columns and filling in missing values if there are any.

In [ ]:
# see df
employer_df.head()

In [ ]:
# Let add full quarter employment and earnings statistics
employer_df = employer_df.merge(full_quarter_df, on = 'employer', how = 'outer')

In [ ]:
# let's not include columns that were just used for calculations
emp = employer_df.drop(['year','quarter','nsep_q2','nsep_q3','nhire_q2','nhire_q3',
                            'num_employed_q2'], axis = 1)

In [ ]:
emp.info()

In [ ]:
# see all nas
emp.isna().sum()

We can see that some variables have missing values. For the separation growth rate and full quarter statistics, we can just fill them in with zero. 

In [ ]:
# convert na's for full quarter to 0
var_fill = ['sep_rate', 'hire_rate', 'full_quarter_num', 'full_quarter_earnings', 'full_quarter_avg_earnings']
emp[var_fill] = emp[var_fill].fillna(0)

In [ ]:
# Checking nulls
emp.isna().sum()

Some employers' NAICS codes in 2013Q3 are also incomplete. To properly capture the industries of these employers, we can see if they have non-missing NAICS codes in surrounding quarters (2013Q2 and 2013Q4), and if so, we can assume that the employer's industry did not change during 2013Q3. There will still be some employers with missing NAICS codes, but this will help to limit that number.

In [ ]:
# get all naics codes for employers who NAICS codes were missing in Q3 but in surrounding quarters
qry = '''
select distinct employer, naics_3_digit
from ada_20_osu.oh_2013q3
where employer in (
select distinct(employer) from ada_20_osu.oh_2013q3 where naics_3_digit is null and quarter = 3) and naics_3_digit is not null
order by employer
'''
naics_df = pd.read_sql(qry, conn)

In [ ]:
# see naics_df
naics_df.head()

Now, we can use the `combine_first()` function to insert the NAICS codes from `naics_df` in surrounding quarters for the employers into the missing NAICS codes in `employer_df`.

In [ ]:
# update naics codes where available
employer_df = employer_df.set_index("employer").combine_first(naics_df.set_index("employer")).reset_index()

If there are still missing values, we can just label them as "missing".

In [ ]:
# fill in missing industry code with "missing"
emp['naics_3_digit'] = emp['naics_3_digit'].fillna('missing')

In [ ]:
# confirm no more null values
emp.info()

In [ ]:
# see all naics codes in emp
emp.naics_3_digit.unique()

<font color=red><h3> Checkpoint 4: Finalize Table  </h3></font> 

Finalize your table for employers in 2013Q2 by addressing missing NAICS codes as well as missing values for some of the other calculated measures.

Now our DataFrame is ready for the unsupervised machine learning model. Before we move on to the  [Unsupervised Machine Learning](Unsupervised_ML.ipynb) notebook, let's use `to_csv` to save it to a `.csv` file so that we can read it from another notebook. 

In the code below, you just need to change `YOURNAME` with your home folder name on ADRF.

In [ ]:
# write into csv for usage in clustering notebook
emp.to_csv('/nfshome/YOURNAME/employer_all_new.csv')

## References

Foster, Ian, Rayid Ghani, Ron S. Jarmin, Frauke Kreuter, and Julia Lane, eds. *Big data and social science: A practical guide to methods and tools.* crc Press, 2016.

Davis, Steven J., and John Haltiwanger. "Gross job creation, gross job destruction, and employment reallocation." *The Quarterly Journal of Economics* 107, no. 3 (1992): 819-863.